In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [9]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
import tensorflow as tf
from glob import glob
import matplotlib.pyplot as plt

In [10]:
dataset_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/'
training_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/train/'
validation_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/val/'
test_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/test/'
model_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/resnet_152v2_model.h5'

IMAGE_SIZE = [224, 224,3]
EPOCHS = 30
TARGET_SIZE = (224,224)
BATCH_SIZE = 16

In [11]:
# useful for getting number of output classes
folders = glob(training_path+'*')
print(len(folders))

4


In [13]:
# importing the resnet and using the imagenet weights
# Here we will be using imagenet weights

resnet_152 = ResNet152V2(include_top=False,weights='imagenet',input_shape=IMAGE_SIZE)

In [14]:
# don't train existing weights
for layer in resnet_152.layers:
  layer.trainable = False

In [15]:
# our layers - you can add more if you want
x_network = Flatten()(resnet_152.output)
# adding the prediction layer
prediction_layer = Dense(len(folders),activation='softmax')(x_network)

In [16]:
# create a model object
model = Model(inputs = resnet_152.input , outputs = prediction_layer)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_______________________________________________________________________________________